In [3]:
library(grpglmnet)
library(grpreg)
library(grplasso)
library(sparsegl)
library(gglasso)
library(microbenchmark)
library(progress)

In [4]:
objective <- function(X, y, groups, group_sizes, beta, lmda, penalty)
{
    reg = sum(sapply(1:length(groups), function(i) penalty[i] * sqrt(sum(beta[(groups[i]+1) : (groups[i]+group_sizes[i])] ** 2))))
    0.5 * sum((y - X %*% beta) ** 2) + lmda * reg
}

In [5]:
generate.data <- function(
    seed,
    n, 
    p,
    snr,
    group_size
)
{
    set.seed(seed)
    X <- matrix(rnorm(n * p), n, p)
    X <- apply(X, 2, scale)
    beta <- runif(p, -1, 1) 
    k <- as.integer(3 * snr)
    beta[1:(p-k)] <- 0
    y <- as.numeric(X %*% beta + rnorm(n))
    y <- y - mean(y)
    n_full_groups <- as.integer(p / group_size)
    groups <- c((0:(n_full_groups-1)) * group_size, p)
    group_sizes <- groups[2:length(groups)] - groups[1:length(groups)-1]
    groups <- groups[1:length(groups)-1]
    list(
        X=X,
        y=y,
        groups=groups,
        group_sizes=group_sizes
    )
}

bench.packages <- function(
    seed=0,
    n=100,
    p=100,
    snr=1,
    group_size=10,
    subset=c("grpreg", "grplasso", "sparsegl", "gglasso")
)
{
    ps <- p
    times <- matrix(NA, length(ps), 5)
    objs <- matrix(NA, length(ps), 5)
    n_lmdas <- rep(NA, length(ps))
    colnames(times) <- c("grpglmnet", "grpreg", "grplasso", "sparsegl", "gglasso")
    colnames(objs) <- colnames(times)
    
    other_fs <- list(
        "grpreg"=function(X,y,group,lmdas,penalty) grpreg(X, y, group, penalty='grLasso', family='gaussian', lambda=lmdas / length(y), eps=1e-7),
        "grplasso"=function(X,y,group,lmdas,penalty) grplasso(X, y, index=as.integer(group), lambda=2*lmdas, center=FALSE, standardize=FALSE, model=LinReg(), control=grpl.control(trace=0)),
        "sparsegl"=function(X,y,group,lmdas,penalty) sparsegl(X, y, group=as.integer(group), family='gaussian', lambda=lmdas / length(y), intercept=FALSE, standardize=FALSE, pf_group=penalty, asparse=0),
        "gglasso"=function(X,y,group,lmdas,penalty) gglasso(X, y, group=as.integer(group), loss='ls', lambda=lmdas / length(y), pf=penalty)
    )

    pb <- txtProgressBar(min = 0,      # Minimum value of the progress bar
                         max = length(ps), # Maximum value of the progress bar
                         style = 3,    # Progress bar style (also available style = 1 and style = 2)
                         width = 50,   # Progress bar width. Defaults to getOption("width")
                         char = "=")   # Character used to create the bar
    for (i in 1:length(ps)) {
        p <- ps[i]

        # generate data
        data <- generate.data(seed, n, p, snr, group_size)
        X <- data$X
        y <- data$y
        groups <- data$groups
        group_sizes <- data$group_sizes
        penalty <- sqrt(group_sizes)
        
        # run mine
        mine_n_times <- ifelse(p <= 2**8, 5L, 1L)
        times[i, 1] <- summary(microbenchmark({grpglmnet.out <- group_basil(X, y, groups, group_sizes, penalty=penalty, method='naive')}, times=mine_n_times, unit='ns'))$min
        lmdas <- grpglmnet.out$lmdas
        n_lmdas[i] <- length(lmdas)

        # prepare input for other packages
        group <- as.factor(rep(1:length(groups), times=group_sizes))

        # run other packages
        other_outs <- list()
        for (ss in subset) {
            times[i, ss] <- summary(microbenchmark({other_outs[[ss]] <- other_fs[[ss]](X, y, group, lmdas, penalty)}, times=1L, unit='ns'))$mean
        }

        # save last objectives
        last_idx <- length(lmdas)
        objs[i, 1] <- objective(X, y, groups, group_sizes, grpglmnet.out$beta[,last_idx], lmdas[last_idx], penalty)
        for (ss in subset) {
            if (ss == "grpreg") {
                if (last_idx <= dim(other_outs[[ss]]$beta)[2]) {
                    objs[i, ss] <- objective(X, y, groups, group_sizes, other_outs[[ss]]$beta[2:nrow(other_outs[[ss]]$beta), last_idx], lmdas[last_idx], penalty)
                }
            } else if (ss == "grplasso") {
                objs[i, ss] <- objective(X, y, groups, group_sizes, other_outs[[ss]]$coefficients[, last_idx], lmdas[last_idx], penalty)
            } else {
                objs[i, ss] <- objective(X, y, groups, group_sizes, other_outs[[ss]]$beta[, last_idx], lmdas[last_idx], penalty)
            }
        }
        
        setTxtProgressBar(pb, i)
    }
    close(pb)
    
    times <- times * 1e-9
    
    list(
        times=times,
        objs=objs,
        n_lmdas=n_lmdas
    )
}

In [6]:
p <- 2 ** (4 : 16)
out.snr_1 <- bench.packages(p=p)

  |==================================================| 100%


In [7]:
p.large <- 2 ** (17 : 18)
out.large.snr_1 <- bench.packages(p=p.large, subset=c("sparsegl", "gglasso", "grpreg"))

  |==================================================| 100%


In [8]:
p.large2 <- 2 ** (19 : 20)
out.large2.snr_1 <- bench.packages(p=p.large2, subset=c("sparsegl", "gglasso"))

  |==================================================| 100%


In [9]:
times.snr_1 <- rbind(out.snr_1$times, out.large.snr_1$times, out.large2.snr_1$times)
objs.snr_1 <- rbind(out.snr_1$objs, out.large.snr_1$objs, out.large2.snr_1$objs)
n_lmdas.snr_1 <- c(out.snr_1$n_lmdas, out.large.snr_1$n_lmdas, out.large2.snr_1$n_lmdas)

In [10]:
write.table(times.snr_1, "data/gl_packages_time_snr_1.csv", sep=',', row.names=FALSE, col.names=FALSE)
write.table(objs.snr_1, "data/gl_packages_obj_snr_1.csv", sep=',', row.names=FALSE, col.names=FALSE)
write.table(n_lmdas.snr_1, "data/gl_packages_n_lmda_snr_1.csv", sep=',', row.names=FALSE, col.names=FALSE)

Now try for SNR = 1/3

In [20]:
p <- 2 ** (4 : 16)
out.snr_03 <- bench.packages(p=p, snr=1/3)

  |==================================================| 100%


In [12]:
p.large <- 2 ** (17 : 18)
out.large.snr_03 <- bench.packages(p=p.large, snr=1/3, subset=c("sparsegl", "gglasso", "grpreg"))

  |==================================================| 100%


In [13]:
p.large2 <- 2 ** (19 : 20)
out.large2.snr_03 <- bench.packages(p=p.large2, snr=1/3, subset=c("sparsegl", "gglasso"))

  |==================================================| 100%


In [21]:
times.snr_03 <- rbind(out.snr_03$times, out.large.snr_03$times, out.large2.snr_03$times)
objs.snr_03 <- rbind(out.snr_03$objs, out.large.snr_03$objs, out.large2.snr_03$objs)
n_lmdas.snr_03 <- c(out.snr_03$n_lmdas, out.large.snr_03$n_lmdas, out.large2.snr_03$n_lmdas)

In [22]:
write.table(times.snr_03, "data/gl_packages_time_snr_03.csv", sep=',', row.names=FALSE, col.names=FALSE)
write.table(objs.snr_03, "data/gl_packages_obj_snr_03.csv", sep=',', row.names=FALSE, col.names=FALSE)
write.table(n_lmdas.snr_03, "data/gl_packages_n_lmda_snr_03.csv", sep=',', row.names=FALSE, col.names=FALSE)

Try for larger $n$ just to see what happens.

In [16]:
n = 10000

In [17]:
p <- 2 ** (4 : 14)
out.snr_1_nl <- bench.packages(n=n, p=p)

  |==================================================| 100%


In [18]:
times.snr_1_nl <- rbind(out.snr_1_nl$times)#, out.large.snr_1_nl$times, out.large2.snr_1_nl$times)
objs.snr_1_nl <- rbind(out.snr_1_nl$objs)#, out.large.snr_1_nl$objs, out.large2.snr_1_nl$objs)
n_lmdas.snr_1_nl <- c(out.snr_1_nl$n_lmdas)#, out.large.snr_1_nl$n_lmdas, out.large2.snr_1_nl$n_lmdas)

In [19]:
write.table(times.snr_1_nl, "data/gl_packages_time_snr_1_nl.csv", sep=',', row.names=FALSE, col.names=FALSE)
write.table(objs.snr_1_nl, "data/gl_packages_obj_snr_1_nl.csv", sep=',', row.names=FALSE, col.names=FALSE)
write.table(n_lmdas.snr_1_nl, "data/gl_packages_n_lmda_snr_1_nl.csv", sep=',', row.names=FALSE, col.names=FALSE)